In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import datetime
import geopy.distance

from sklearn.mixture import BayesianGaussianMixture
from sklearn.cluster import KMeans

In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
submission_df = pd.read_csv('sample_submission.csv')

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
train_df.shape

In [ ]:
test_df.shape

# Column meaning

|Column | Meaning
|---|---|
|tripid  |Trip ID|
|additional_fare |  Additional fare charged in rupees|
|duration| Duration of the trip in seconds from trip-start to trip-end|
|meter_waiting| Duration in seconds indicating the total time the vehicle was immobile from trip-start to trip-end|
|meter_waiting_fare| Fare for the time the vehicle was stopped within trip-start to trip-end duration. This may be due to traffic or other reason ex: stop at supermarket|
|meter_waiting_till_pickup| Time between trip-accept to trip-start|
|pickup_time| Date and time at pickup|
|drop_time| Date and time at drop off|
|pick_lat| Pickup latitude|
|pick_lon| Pickup longitude|
|drop_lat| Drop latitude|
|drop_lon| Drop longitudefare Trip fare|
|label| Label indicating if it is a correct or incorrect trip fare|


## Check missing values

In [ ]:
train_df.isna().any()

In [ ]:
train_df.isna().sum()

Missing values for additional_fare, duration, meter_waiting, meter_waiting_fare, meter_waiting_till_pickup are in the same row. Probably intentially dropped?

In [ ]:
train_df[train_df['additional_fare'].isna() & train_df['duration'].isna() & train_df['meter_waiting'].isna() & train_df['meter_waiting'].isna() & train_df['meter_waiting_fare'].isna() & train_df['meter_waiting_till_pickup'].isna()]

Most of the missing values for fair have missing values for other missing column however few have values for other missing columns 

In [ ]:
train_df[train_df['additional_fare'].isna() & train_df['duration'].isna() & train_df['meter_waiting'].isna() & train_df['meter_waiting'].isna() & train_df['meter_waiting_fare'].isna() & train_df['meter_waiting_till_pickup'].isna() & train_df['fare'].isna()]

In [ ]:
train_df[~(train_df['additional_fare'].isna() & train_df['duration'].isna() & train_df['meter_waiting'].isna() & train_df['meter_waiting'].isna() & train_df['meter_waiting_fare'].isna() & train_df['meter_waiting_till_pickup'].isna()) & train_df['fare'].isna()]

In [ ]:
test_df.isna().any()

# Label distribution

In [ ]:
sns.countplot(x='label', data=train_df)

### Distribution for missing values

In [ ]:
sns.countplot(x='label', data=train_df[train_df['additional_fare'].isna() | train_df['fare'].isna()])

### Distribution after dropping missing values

In [ ]:
sns.countplot(x='label', data=train_df.dropna())

# Check for data leaks

In [ ]:
pd.merge(train_df, test_df, on=['tripid'],how='inner')

# Feature extraction functions

In [ ]:
def extract_time(feature='date'):
    def f(time_stamp): 
        date,time = time_stamp.strip().split()
        date = list(map(int, date.split('/')))
        time = list(map(int, time.split(':')))
        if feature == 'date':
            return date[1]
        if feature == 'month':
            return date[0]
        if feature == 'year':
            return date[2]
        if feature == 'hour':
            return time[0]
        if feature == 'minute':
            return time[1]
    return f
        

In [ ]:
def get_combined_dataset(cols, train=train_df, test=test_df):
    tmp_1 = train.loc[:,cols]
    tmp_1['dataset'] = 'train'

    tmp_2 = test.loc[:,cols]
    tmp_2['dataset'] = 'test'
    return tmp_1.append(tmp_2)
    

# Data distribution

In [ ]:
data = train_df.dropna().drop(['tripid'],axis=1)
data.head()

### Extract time data

In [ ]:
data['pickup_month'] = data['pickup_time'].map(extract_time('month'))
data['pickup_date'] = data['pickup_time'].map(extract_time('date'))
data['pickup_year'] = data['pickup_time'].map(extract_time('year'))
data['pickup_hour'] = data['pickup_time'].map(extract_time('hour'))
data['pickup_minute'] = data['pickup_time'].map(extract_time('minute'))

In [ ]:
data['drop_month'] = data['drop_time'].map(extract_time('month'))
data['drop_date'] = data['drop_time'].map(extract_time('date'))
data['drop_year'] = data['drop_time'].map(extract_time('year'))
data['drop_hour'] = data['drop_time'].map(extract_time('hour'))
data['drop_minute'] = data['drop_time'].map(extract_time('minute'))

In [ ]:
test_df['pickup_month'] = test_df['pickup_time'].map(extract_time('month'))
test_df['pickup_date'] = test_df['pickup_time'].map(extract_time('date'))
test_df['pickup_year'] = test_df['pickup_time'].map(extract_time('year'))
test_df['pickup_hour'] = test_df['pickup_time'].map(extract_time('hour'))
test_df['pickup_minute'] = test_df['pickup_time'].map(extract_time('minute'))

test_df['drop_month'] = test_df['drop_time'].map(extract_time('month'))
test_df['drop_date'] = test_df['drop_time'].map(extract_time('date'))
test_df['drop_year'] = test_df['drop_time'].map(extract_time('year'))
test_df['drop_hour'] = test_df['drop_time'].map(extract_time('hour'))
test_df['drop_minute'] = test_df['drop_time'].map(extract_time('minute'))

## Data correlations

In [ ]:
data.describe()

In [ ]:
data.corr()

In [ ]:
corr = data.corr()
# fig, ax = plt.subplots(figsize=(10,10))
# plt.xticks(range(len(corr.columns)), corr.columns)
# plt.yticks(range(len(corr.columns)), corr.columns)
# ax.matshow(corr)
sns.heatmap(corr)

## Lattitude longitude data

In [ ]:
sns.scatterplot(x='pick_lat',y='pick_lon',data=data,hue='label')

In [ ]:
sns.scatterplot(x='drop_lat',y='drop_lon',data=data,hue='label')

In [ ]:
sns.scatterplot(x='drop_lat',y='drop_lon',data=data[data['drop_lat'] < 30],hue='label')

One coordinate seems to be an outlier

In [ ]:
data = data[data['drop_lat'] < 30]

#### Comparision of training and testing sets

In [ ]:
cols = ['drop_lat', 'drop_lon', 'pick_lat', 'pick_lon']
comb_data = get_combined_dataset(cols, train=data)

In [ ]:
sns.scatterplot(x='pick_lat',y='pick_lon',data=comb_data,hue='dataset')

In [ ]:
sns.scatterplot(x='drop_lat',y='drop_lon',data=comb_data,hue='dataset')

## Time data

In [ ]:
sns.scatterplot(x='pickup_date',y='drop_date',data=data, hue='label')

In [ ]:
data[data['pickup_date'] != data['drop_date']]

In [ ]:
test_df[test_df['pickup_date'] != test_df['drop_date']]

There are instances of trip taking more than a single date in both test and train df

In [ ]:
sns.countplot(x='label',data=data[data['pickup_date'] != data['drop_date']])

In [ ]:
def is_more_than_one_day(row):
    return 1 if row['pickup_date'] != row['drop_date'] else 0

In [ ]:
data['is_more_than_one_day'] = data.apply(is_more_than_one_day,axis=1)
test_df['is_more_than_one_day'] = test_df.apply(is_more_than_one_day,axis=1)

### Time location relations

In [ ]:
sns.scatterplot(x='pick_lat',y='pick_lon',data=data,hue='pickup_month')

In [ ]:
sns.scatterplot(x='pick_lat',y='pick_lon',data=data,hue='pickup_date')

In [ ]:
sns.scatterplot(x='pick_lat',y='pick_lon',data=data,hue='pickup_date')

### Time distribution accross datasets

In [ ]:
cols = ['pickup_month',
       'pickup_date', 'pickup_year', 'pickup_hour', 'pickup_minute',
       'drop_month', 'drop_date', 'drop_year', 'drop_hour', 'drop_minute']
comb_data = get_combined_dataset(cols, train=data)

In [ ]:
sns.countplot(x='pickup_month', data=comb_data, hue='dataset')

In [ ]:
sns.countplot(x='drop_month', data=comb_data, hue='dataset')

In [ ]:
sns.countplot(x='pickup_year', data=comb_data, hue='dataset')

In [ ]:
sns.countplot(x='pickup_date', data=comb_data, hue='dataset')

In [ ]:
sns.countplot(x='drop_date', data=comb_data, hue='dataset')

## Continious value distribution

In [ ]:
# sns.pairplot(data=data,hue='label')

In [ ]:
sns.distplot(data[data['label']=='correct']['fare'], hist=False)
sns.distplot(data[data['label']=='incorrect']['fare'], hist=False)

In [ ]:
sns.distplot(data[data['label']=='correct']['additional_fare'], hist=False)
sns.distplot(data[data['label']=='incorrect']['additional_fare'], hist=False)

In [ ]:
sns.distplot(data[data['label']=='correct']['meter_waiting'], hist=False)
sns.distplot(data[data['label']=='incorrect']['meter_waiting'], hist=False)

In [ ]:
sns.distplot(data[data['label']=='correct']['meter_waiting_fare'], hist=False)
sns.distplot(data[data['label']=='incorrect']['meter_waiting_fare'], hist=False)

In [ ]:
sns.distplot(data[data['label']=='correct']['meter_waiting_till_pickup'], hist=False)
sns.distplot(data[data['label']=='incorrect']['meter_waiting_till_pickup'], hist=False)

In [ ]:
sns.distplot(data[data['label']=='correct']['duration'], hist=False)
sns.distplot(data[data['label']=='incorrect']['duration'], hist=False)

# Clustering lat longs

In [ ]:
sns.scatterplot(x='pick_lat',y='pick_lon',data=data,hue='label')

In [ ]:
sns.scatterplot(x='drop_lat',y='drop_lon',data=data,hue='label')

In [ ]:
cols = ['pick_lat','pick_lon']
comb_data = get_combined_dataset(cols, train=data)

In [ ]:
gmm_pick = BayesianGaussianMixture(n_components=3)
gmm_pick.fit(comb_data[cols].values)

In [ ]:
data['pick_cluster'] = gmm_pick.predict(data[cols].values)
test_df['pick_cluster'] = gmm_pick.predict(test_df[cols].values)

In [ ]:
cols = ['drop_lat','drop_lon']
comb_data = get_combined_dataset(cols, train=data)

gmm_drop = BayesianGaussianMixture(n_components=3,max_iter=1000)
gmm_drop.fit(comb_data[cols].values)

data['drop_cluster'] = gmm_drop.predict(data[cols].values)
test_df['drop_cluster'] = gmm_drop.predict(test_df[cols].values)

In [ ]:
sns.scatterplot(x='pick_lat',y='pick_lon',data=data,hue='pick_cluster')

In [ ]:
sns.scatterplot(x='drop_lat',y='drop_lon',data=data,hue='drop_cluster')

# Distance features

In [ ]:
def calculate_trip_distance(row):
    coords_1 = (row['pick_lat'],row['pick_lon'])
    coords_2 = (row['drop_lat'],row['drop_lon'])
    return geopy.distance.geodesic(coords_1, coords_2).km

In [ ]:
data['distance_km'] = data.apply(calculate_trip_distance,axis=1).clip(0,100)
test_df['distance_km'] = test_df.apply(calculate_trip_distance,axis=1).clip(0,100)

In [ ]:
data['distance_km'].plot()

In [ ]:
test_df['distance_km'].plot()

In [ ]:
data[data['label'] == 'correct']['distance_km'].plot()

In [ ]:
data[data['label'] == 'correct']['distance_km'].describe()

In [ ]:
data[data['label'] == 'incorrect']['distance_km'].plot()

In [ ]:
data[data['label'] == 'incorrect']['distance_km'].describe()

# Fare features

In [ ]:
data['fare'].describe()

In [ ]:
test_df['fare'].describe()

In [ ]:
data[data['fare'] > 400].label

In [ ]:
sns.countplot(x='label',data=data[data['fare'] > 1000])

In [ ]:
def fare_per_distance(row):
    return row['fare'] / (row['distance_km']+0.01)

In [ ]:
data['fare_per_km'] = data.apply(fare_per_distance,axis=1)
test_df['fare_per_km'] = test_df.apply(fare_per_distance,axis=1)

In [ ]:
data[data['label'] == 'incorrect']['fare_per_km'].plot()

In [ ]:
data[data['label'] == 'correct']['fare_per_km'].plot()

# Time based features

In [ ]:
data['pickup_hour'].describe()

In [ ]:
data['drop_minute'].describe()

In [ ]:
def time_slot(row,by='pickup'):
    hour = row[f'{by}_hour']
    if 7 <= hour <= 9:
        return 1
    if 12 <= hour <= 2:
        return 2
    if 4 <= hour <= 6:
        return 3
    return 0

In [ ]:
data['pickup_timeslot'] = data.apply(time_slot,axis=1)
test_df['pickup_timeslot'] = test_df.apply(time_slot,axis=1)

In [ ]:
(2017, 10, 20)

In [ ]:
def day_of_week(row,by='pickup'):
    date = row[f'{by}_date']
    month = row[f'{by}_month']
    year = row[f'{by}_year']
    d = datetime.datetime(year,month,date).weekday()
    return d

In [ ]:
data['day_of_week'] = data.apply(day_of_week,axis=1)
test_df['day_of_week'] = test_df.apply(day_of_week,axis=1)

In [ ]:
def is_weekday(row,by='pickup'):
    date = row['day_of_week']
    return 1 if date < 5 else 0

In [ ]:
data['is_weekday'] = data.apply(is_weekday,axis=1)
test_df['is_weekday'] = test_df.apply(is_weekday,axis=1)

In [ ]:
def cal_time_difference(row):
    pickup_date = row['pickup_date']
    pickup_month = row['pickup_month']
    pickup_year = row['pickup_year']
    pickup_hour = row['pickup_hour']
    pickup_minute = row['pickup_minute']
    pickup_time = datetime.datetime(pickup_year, pickup_month, pickup_date, pickup_hour, pickup_minute)
    
    drop_date = row['drop_date']
    drop_month = row['drop_month']
    drop_year = row['drop_year']
    drop_hour = row['drop_hour']
    drop_minute = row['drop_minute']
    drop_time = datetime.datetime(drop_year, drop_month, drop_date, drop_hour, drop_minute)
    
    delta = drop_time - pickup_time
    return delta.seconds - row['duration']

In [ ]:
data['cal_time_difference'] = data.apply(cal_time_difference,axis=1)
test_df['cal_time_difference'] = test_df.apply(cal_time_difference,axis=1)

In [ ]:
data['cal_time_difference'].describe()

In [ ]:
sns.distplot(data[data['label']=='correct']['cal_time_difference'], hist=False)
sns.distplot(data[data['label']=='incorrect']['cal_time_difference'], hist=False)

# Save the data frames

In [ ]:
def encode_label(label):
    if label == 'correct':
        return 1
    elif label == 'incorrect':
        return 0
    else:
        return label

In [ ]:
data['label'] = data['label'].map(encode_label)

In [ ]:
data.columns

In [ ]:
columns_to_save = [
    'additional_fare', 
    'duration', 
    'meter_waiting', 
    'meter_waiting_fare',
    'meter_waiting_till_pickup', 
    'fare',
    'pickup_date', 
    'pickup_hour', 
    'pickup_minute',
    'drop_date', 
    'drop_hour', 
    'drop_minute',
    'pick_cluster',
    'is_more_than_one_day',
    'distance_km',
    'fare_per_km',
    'pickup_timeslot',
    'day_of_week',
    'is_weekday',
    'cal_time_difference']
data.loc[:, columns_to_save+['label']].to_csv('train_df.csv',index=False)
test_df.loc[:, columns_to_save].to_csv('test_df.csv',index=False)